# Italian towns analysis for tourism

    
         
## Executive summary         

A **tour operator** would like to have a data driven approach when suggesting destinations in Italy.

Specifically, given the incredible variability of Italy, they asked for some kind of **clustering** approach, that might be able to suggest similar or different locations.

Hopefully, an added value would be given by some **new insights** provided by data analysis.

The problem is studied on the basis of data from Foursquare. The approach is based on the assumption that even if Italy is a mix of many traditions and cultures, there exists *a common denominator defined by food and wine culture*.

**The results of this analysis show that it is possible to categorise interesting features of Italian towns on the basis of the food and wine venues that they offer.**

### Importing required libraries
     
First of all we import all the libraries that will be requried for the analysis

In [97]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

## Data source
     
Data used in this project are taken from Foursquare.     
So we have to provide credentials to access Forsquare services:

In [7]:
CLIENT_ID = 'W00YRBTATRWAQFHU0S5LFKNRF2ETKRLZDI1Q0ERHTX0IMRD2' # your Foursquare ID
CLIENT_SECRET = 'FW1OZXNNFEGEEAGM4Y5O02MISZLGAPRI2G5QBKYCZVJYD0LY' # your Foursquare Secret
VERSION = '20200508' # Foursquare API version

Your credentails:
CLIENT_ID: W00YRBTATRWAQFHU0S5LFKNRF2ETKRLZDI1Q0ERHTX0IMRD2
CLIENT_SECRET:FW1OZXNNFEGEEAGM4Y5O02MISZLGAPRI2G5QBKYCZVJYD0LY


The following function, provided by Foursquare, is used to extract venues categories from the data stream obtained from Foursquare:

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

We will focus on some famous Italian towns, which cover the country North to South. here is the list:

In [231]:
towns=['Trento, Trentino-Alto Adige, Italy','Venice, Veneto, Italy','Milan, Lombardy, Italy','Rome, Latium, Italy',
       'florence','Catania, Sicily, Italy','Trieste, Friuli Venezia Giulia, Italy',
       'Udine, Friuli Venezia Giulia, Italy','Bologna, Emilia-Romagna, Italy','Pisa, Tuscany, Italy',
      'Verona, Veneto, Italy','Padua, Veneto, Italy','Naples, Campania, Italy','Bari, Apulia, Italy','Amalfi, Campania, Italy']


We create two empty frames that will be populated with the results coming from Forsquare: 

In [233]:
all_venues = pd.DataFrame(columns=['town','town_lat','town_lng','name','categories','lat','lng'])
occ = pd.DataFrame(columns=['town','q','nbr'])

Now it's time to get the data from Foursquare. Here we have two queries, one for "restaurant" and the other for "wine", that are looped over the towns used in the analysis. The results of these queries will populate our thw data frames: 

In [234]:
for i in towns:
    address = i
    geolocator = Nominatim(user_agent="my_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    radius=1000
    LIMIT=500
    
    town_name = i.split(",")
    town_name = town_name[0]
    
    SQ = ['restaurant','wine']
    
    for search_query in SQ:
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            search_query,
            radius, 
            LIMIT)
    
        results = requests.get(url).json()
    
        venues = results['response']['groups'][0]['items']
        nearby_venues = json_normalize(venues) # flatten JSON
        
        if len(nearby_venues)>0:
            
            filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
            nearby_venues =nearby_venues.loc[:, filtered_columns]
            nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
            nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
            
            town = [town_name]*len(nearby_venues)
            town_lat = [latitude]*len(nearby_venues)
            town_lng = [longitude]*len(nearby_venues)
            
            data = {'town':town, 'town_lat':town_lat, 'town_lng':town_lng}
            out = pd.DataFrame(data)
            
            out['name'] = nearby_venues['name']
            out['categories'] = nearby_venues['categories']
            out['lat'] = nearby_venues['lat']
            out['lng'] = nearby_venues['lng']
            
            all_venues = all_venues.append(out,ignore_index=True)
            
        d2 = {'town':[town_name], 'q':[search_query], 'nbr':[len(nearby_venues)]}
        o2 = pd.DataFrame(d2)
        
        occ = occ.append(o2,ignore_index=True)
    
      

Let's take a look at all_venues, which contains the detailed venues:

In [249]:
all_venues.head()

,town,town_lat,town_lng,name,categories,lat,lng
0,Trento,46.066423,11.12576,Il posto di Ste,Sandwich Place,46.068258,11.125384
1,Trento,46.066423,11.12576,Tipico,Restaurant,46.067067,11.124574
2,Trento,46.066423,11.12576,Uva & Menta,Italian Restaurant,46.065531,11.124665
3,Trento,46.066423,11.12576,Bar Plan,Sandwich Place,46.068385,11.124931
4,Trento,46.066423,11.12576,Scrigno del Duomo,Trattoria/Osteria,46.067449,11.121270


Next, have alook at occ, standing for "occurrencies", which contains the numerosity of restaurant and wine results for each single town.     
We have to pivot the data frame, because originally every town appears twice (just visualisation, due to the append approach used to get data in the loop:

In [236]:
occ=occ.pivot(index='town',columns='q',values='nbr')
occ.reset_index(level=0, inplace=True)
occ

q,town,restaurant,wine
0,Amalfi,49,11
1,Bari,70,16
2,Bologna,100,100
3,Catania,80,35
4,Milan,100,100
5,Naples,100,26
6,Padua,46,26
7,Pisa,100,40
8,Rome,93,13
9,Trento,99,11


## Data preparation

Now we create dummy variables and compute mean values for occurrencies. Note that for example, "America Restaurant", which is extremely unusual in Italy, appears only in Milan, Venice and Florence, which are the most touristic towns:

In [237]:
# one hot encoding
dt_onehot = pd.get_dummies(all_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['town'] = all_venues['town'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

In [238]:
dt_grouped = dt_onehot.groupby('town').mean().reset_index()

In [239]:
dt_grouped

,town,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Butcher,Cafeteria,Café,Campanian Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Creperie,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Eastern European Restaurant,Emilia Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Friterie,Friuli Restaurant,Gastropub,German Restaurant,Gourmet Shop,Greek Restaurant,Hawaiian Restaurant,Hostel,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Korean Restaurant,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Noodle House,Persian Restaurant,Piadineria,Pizza Place,Puglia Restaurant,Restaurant,Romagna Restaurant,Roman Restaurant,Salad Place,Sandwich Place,Sardinian Restaurant,Seafood Restaurant,Sicilian Restaurant,Snack Place,Spanish Restaurant,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Trattoria/Osteria,Trentino Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veneto Restaurant,Wine Bar,Wine Shop,Winery
0,Amalfi,0.000,0.000,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.00000,0.000000,0.000,0.000000,0.100000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.016667,0.033333,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.500000,0.000000,0.000,0.000000,0.000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.083333,0.000,0.000000,0.000000,0.016667,0.000,0.050000,0.000000,0.033333,0.000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bari,0.000,0.000,0.000000,0.000000,0.023256,0.011628,0.023256,0.000000,0.000000,0.00000,0.011628,0.000,0.011628,0.197674,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000,0.046512,0.000000,0.000000,0.000000,0.000,0.000000,0.011628,0.000000,0.011628,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,0.000,0.011628,0.000,0.000,0.000000,0.000000,0.162791,0.011628,0.000,0.000000,0.000,0.000000,0.023256,0.000000,0.000000,0.011628,0.000000,0.000000,0.000000,0.174419,0.011628,0.046512,0.000,0.000000,0.000000,0.011628,0.000,0.081395,0.000000,0.023256,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,0.046512,0.000000,0.011628
2,Bologna,0.000,0.005,0.000000,0.000000,0.010000,0.005000,0.020000,0.000000,0.000000,0.00000,0.010000,0.005,0.000000,0.100000,0.000000,0.005000,0.015000,0.01,0.000000,0.000000,0.000000,0.000,0.000,0.020000,0.000000,0.000000,0.000000,0.035,0.005000,0.000000,0.000000,0.000000,0.010,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.005,0.005000,0.000,0.000,0.010000,0.005000,0.335000,0.005000,0.005,0.000000,0.000,0.005000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005000,0.055000,0.000000,0.030000,0.005,0.000000,0.000000,0.025000,0.000,0.025000,0.000000,0.005000,0.000,0.000000,0.000000,0.000000,0.000000,0.015000,0.000000,0.000000,0.000000,0.020000,0.000000,0.130000,0.010000,0.015000
3,Catania,0.000,0.000,0.000000,0.008696,0.000000,0.000000,0.017391,0.000000,0.008696,0.00000,0.043478,0.000,0.000000,0.086957,0.000000,0.000000,0.034783,0.00,0.000000,0.008696,0.000000,0.000,0.000,0.008696,0.000000,0.000000,0.000000,0.000,0.000000,0.034783,0.000000,0.008696,0.000,0.008696,0.000,0.008696,0.000000,0.000000,0.000000,0.008696,0.000000,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.173913,0.000000,0.000,0.000000,0.000,0.000000,0.043478,0.008696,0.000000,0.000000,0.000000,0.000000,0.008696,0.130435,0.000000,0.

Now we define a function that we are going to use to create a human readable visualisation of the 10 most common venues for each town respectively:

In [240]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [241]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['town']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['town'] = dt_grouped['town']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amalfi,Italian Restaurant,Café,Restaurant,Trattoria/Osteria,Seafood Restaurant,Mediterranean Restaurant,Snack Place,Food,Pizza Place,Bistro
1,Bari,Café,Pizza Place,Italian Restaurant,Seafood Restaurant,Wine Bar,Diner,Restaurant,Snack Place,Bakery,Bistro
2,Bologna,Italian Restaurant,Wine Bar,Café,Pizza Place,Emilia Restaurant,Restaurant,Seafood Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Bistro
3,Catania,Italian Restaurant,Pizza Place,Café,Restaurant,Wine Bar,Seafood Restaurant,Mediterranean Restaurant,Burger Joint,Cocktail Bar,Fast Food Restaurant
4,Milan,Italian Restaurant,Café,Wine Bar,Pizza Place,Sandwich Place,Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,Sushi Restaurant
5,Naples,Italian Restaurant,Pizza Place,Trattoria/Osteria,Café,Wine Bar,Diner,Seafood Restaurant,Mediterranean Restaurant,Restaurant,Chinese Restaurant
6,Padua,Italian Restaurant,Café,Wine Bar,Pizza Place,Wine Shop,Bakery,Restaurant,Trattoria/Osteria,Burger Joint,Fast Food Restaurant
7,Pisa,Italian Restaurant,Pizza Place,Café,Sandwich Place,Wine Bar,Restaurant,Indian Restaurant,Gastropub,Fast Food Restaurant,Japanese Restaurant
8,Rome,Café,Italian Restaurant,Pizza Place,Wine Bar,Japanese Restaurant,Burger Joint,Restaurant,Seafood Restaurant,Fast Food Restaurant,Diner
9,Trento,Café,Italian Restaurant,Pizza Place,Trattoria/Osteria,Sandwich Place,Wine Bar,Restaurant,Sushi Restaurant,Bakery,Gastropub


This visualisation is not so useful, because we cannot have a full picture just looking at it. We are going then to use a machine learning approach to find out some unsueprvised insight. 

## Methodology

We will use K Means clustering algorithm from Sklearn.
     
The number of clusters is set to three because the analyst thinks that 3 distinct clusters could be interpreted according to a previous hypotesis.    
Yet, other numerosities were studied, with 2 or 4 clusters and 3 confirmed to be most reasonable. 2 was not enough and 4 was confirming the results seen with 3, plus adding some noise that was not easily interpretable.

## Analysis

In [242]:
# set number of clusters
kclusters = 3

dt_grouped_clustering = dt_grouped.drop('town', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

In [243]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [244]:
# add clustering labels
#neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

## Results
    
The clustering algorithm returned an interesting labeling of towns:

In [245]:
dt_merged = occ

dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('town'), on='town')

In [255]:
dt_merged[['town','Cluster Labels']]
#dt_merged

,town,Cluster Labels
0,Amalfi,1
1,Bari,0
2,Bologna,1
3,Catania,0
4,Milan,1
5,Naples,2
6,Padua,0
7,Pisa,2
8,Rome,0
9,Trento,0


## Discussion

**The three clusters can be interpreted** on the basis of good knowledge of Italy.      
*This result is really interesting* because it demonstrates that yes, **it is possible to catagorise Italian towns on the basis of their food venues**.    
Let's discover how and why...

### Cluster 1 : Very turistic and international
     
All of the towns in cluster 1 are very common destinations for turists. These towns, due to their turistic relevance, have lost some of their traditional identity, they have acquired many international features. So, even if you can find *also* traditional restaurants and wines, you can easily find food that is not original from that specific place. For an Italian changing food traditions is really a relevant step. So we can claim that these towns, known for tourism, have also acquired many international features, which do not belong to the place itself and this is well seen from restaurant and wine venues.

      
### Cluster 2 : Very turistic and traditional
     
Cluster 2 is made up by Pisa and Napoli, two towns that are very well known by turists. Yet these two towns are very bound to their traditions and thus are less international. In these places you will find many turists, but local traditions are well kept and preserved. 
     
     
### Cluster 0 : Beautiful and very traditional
     
In this cluster all of the towns, with the important exception of Rome, are not very known by turists. These towns are all extremely beautiful and rich with history and art. For these towns, being out of the main turistic trips, is the key to preserve their traditions. Here trditions are **very** well kept and preserved. Rome is an interesting case: Rome is one of the main destinations in the world, but her traditions are so strong that they are almost untouched and surely very well kept and preserved, like in the other towns of this cluster.

## Conclusion

As it was supposed, it is possible to label Italian towns on the basis of the food venues that they offer.
     
This is made possible by the Italian culture for food and wine. When an area changes the food traditions, it means that something is changing. Such a change we have seen that, on the basis of domain experience, can be related to the level of globalisation. Globalisation in Italy is often brought by turism, in order to satisfy turistis desires.
     
For example, the famous lasagne are from Bologna, you cannot expect to eat lasagne in Trento, but you will find lasagne both in Florence and Venice.
     
But now you know that before eating something in italy you should look for traditional recipes of the specific town and look only for those restaurants that offer only traditional food of the specific place. The other restaurants are for tourists....! ;)